In [1]:
from peewee import (
    SqliteDatabase, JOIN, 
    Model, Field, 
    AutoField, ForeignKeyField, 
    TextField, FloatField, 
    IntegerField, BooleanField, 
    TimeField
)

In [2]:
db = SqliteDatabase("test_db.db", pragmas = { "foreign_keys" : 1, "journal_mode": "wal" })

In [3]:
class ListField(Field):
    field_type = "list"

    def db_value(self, value):
        return str(value)

    def python_value(self, value):
        pval = []

        for entry in value[1 : -1].split(","):
            try:
                pval.append(float(entry))

            except:
                pval.append(entry.strip().replace("'", ""))

        return pval

In [4]:
class Structure(Model):
    structure_id = AutoField()

    type = TextField()
    direction = ListField()
    theta = FloatField()

    r0 = FloatField(null = True)
    L = FloatField(null = True)
    radius = FloatField(null = True)

    filletsEnabled = BooleanField(null = True)
    fillets = FloatField(null = True)
    #path = TextField()
    
    class Meta:
        database = db
        db_table = "structures"


class Mesh(Model):
    mesh_id = AutoField()
    structure_id = ForeignKeyField(Structure, backref = "meshes")

    maxSize = FloatField(null = True) 

    class Meta:
        database = db
        db_table = "meshes"
        depends_on = Structure

In [5]:
db.create_tables([Structure, Mesh])

In [22]:
import numpy

In [20]:
s1 = Structure(type = "simple", direction = [1, 0, 0], theta = 0.01)
s1.save()

m1 = Mesh(structure_id = s1, maxSize = 1e-2)
m1.save()

#db.commit()

1

In [19]:
s1.theta = 0.12
s1.save()

1

In [23]:
ss = numpy.array([
    Structure(type = "simple", direction = [1, 0, 0], theta = 0.01),
    Structure(type = "simple", direction = [1, 0, 0], theta = 0.02)
])

In [24]:
ss.dtype

dtype('O')

In [25]:
Structure.bulk_create(ss)

In [26]:
for row in Structure.select():
    print(row.structure_id, row.type, row.direction, row.theta)

1 simple [1.0, 0.0, 0.0] 0.12
2 simple [1.0, 0.0, 0.0] 0.01
3 simple [1.0, 0.0, 0.0] 0.01
4 simple [1.0, 0.0, 0.0] 0.01
5 simple [1.0, 0.0, 0.0] 0.01
6 simple [1.0, 0.0, 0.0] 0.01
7 simple [1.0, 0.0, 0.0] 0.01
8 simple [1.0, 0.0, 0.0] 0.01
9 simple [1.0, 0.0, 0.0] 0.01
10 simple [1.0, 0.0, 0.0] 0.01
11 simple [1.0, 0.0, 0.0] 0.01
12 simple [1.0, 0.0, 0.0] 0.02


In [10]:
sel = Structure.select()[-1]
sel2 = sel.meshes.select()[-1]

In [11]:
sel2

<Mesh: 8>

In [16]:
sel3 = Structure.select().dicts().get()

In [18]:
sel3

{'structure_id': 1,
 'type': 'simple',
 'direction': [1.0, 0.0, 0.0],
 'theta': 0.12,
 'r0': None,
 'L': None,
 'radius': None,
 'filletsEnabled': None,
 'fillets': None}

In [30]:
from anisotropy.core.main import Anisotropy
model = Anisotropy()
params_: list = model.loadFromScratch("test_anisotropy.toml")

In [31]:
len(params_)

171

In [32]:
params = numpy.array(params_)

In [40]:
from pandas import Series

In [41]:
pparams = Series(params_)

In [51]:
test = numpy.array([], dtype = object)

In [52]:
test += 1

In [55]:
?test.fill

Docstring:
a.fill(value)

Fill the array with a scalar value.

Parameters
----------
value : scalar
    All elements of `a` will be assigned this value.

Examples
--------
>>> a = np.array([1, 2])
>>> a.fill(0)
>>> a
array([0, 0])
>>> a = np.empty(2)
>>> a.fill(1)
>>> a
array([1.,  1.])
Type:      builtin_function_or_method
